In [1]:
from pyspark.sql import SparkSession

bucket = "onedayproject"  

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1644247461037_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
trips_df = spark.read.load("s3://"+ bucket +"/tlc_tripsdata/yellow_tripdata_2021-01.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
trips_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|          2.1|         1|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.0|         0.0|                  0.3

In [4]:
trips_df.createOrReplaceTempView("trips_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
trips_silver_df = spark.sql("""
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips_view
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-01-31'
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
sample_df = spark.sql("""
SELECT *
FROM 
    trips_view
LIMIT 10
""")
                      

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
trips_silver_df.createOrReplaceTempView("trips_silver_df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
train_df, test_df = trips_silver_df.randomSplit([0.8, 0.2], seed=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
data_dir = "s3://"+ bucket +"/tlc_tripsdata/"
train_df.coalesce(1).write.format("parquet").mode('overwrite').save(f"{data_dir}/train/")
test_df.coalesce(1).write.format("parquet").mode('overwrite').save(f"{data_dir}/test/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# sample_df = spark.read.parquet(f"{data_dir}/train/*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark.sql("show tables").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|        |trips_silver_df|       true|
|        |     trips_view|       true|
+--------+---------------+-----------+

In [23]:
%%sql

select * from trips_silver_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [7]:
%%sql

select count(*) from trips_silver_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

# DB에 write

In [11]:
jdbc_hostname = ""
jdbc_database = "dm"
username=""
password=""
jdbc_port = 3306
jdbc_url = "jdbc:mysql://{0}:{1}/{2}?characterEncoding=UTF-8".format(jdbc_hostname, jdbc_port, jdbc_database)

# sample_df.write \
#     .format("jdbc").option("url", f"{jdbc_url}") \
#     .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "trips_silver_df") \
#     .option("user", f"{username}").option("password", f"{password}").mode("append").save()

trips_silver_df.write \
    .format("jdbc").option("url", f"{jdbc_url}") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "trips_silver_df") \
    .option("user", f"{username}").option("password", f"{password}").mode("append").save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# DB data Load

In [ ]:
sql="(select * from dm.trips_silver_df limit 100) a"
load_df = spark.read.format("jdbc").option("driver","com.mysql.jdbc.Driver" ).option("url", jdbcUrl) \
    .option("dbtable", sql).option("user", username).option("password", password).load()

load_df.show()

# 실행 취소 = Clear Outputs

In [13]:
from pyspark import SQLContext

sqlContext.cacheTable("trips_silver_df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
%%sql

select * from trips_silver_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()